In [1]:
# grabbing the ntyimesarticle library
from nytimesarticle import articleAPI
api = articleAPI('8da1f056b9bc4a5fb5b3d59fd1c81b20')

In [52]:
# imports needed to extract article text from urls
import requests
import datetime
from newspaper import Article

In [53]:
# data handlign imports
import pandas
import csv

In [115]:
# generate dates with 30 day intervals in the format that nytimes api accepts
begin = '2015-01-03'
finish = '2015-05-10'
start = datetime.datetime.strptime(begin, '%Y-%m-%d')
end = datetime.datetime.strptime(finish, '%Y-%m-%d')

step = datetime.timedelta(days=30)

dates = []

while start <= end:
    date = str(start.date()).replace('-','')
    dates.append(date)
    start += step

In [116]:
# store all links in the list
web_links = []
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

In [117]:
# loop to grab links related to topic in 'q' given the range of dates
for i in range(len(dates)-1):
    print("now considering: " + str(dates[i]) + " to " + str(dates[i+1]))
    params = {
    'api-key': '8da1f056b9bc4a5fb5b3d59fd1c81b20',
    'q': 'movies',
    'begin_date': str(dates[i]),
    'end_date': str(dates[i+1])
    }
    
    resp = requests.get(url=url, params=params)
    data = resp.json()
    
    for response in data:
        if response == 'response':
            for docs in data[response]:
                if docs == 'docs':
                    for item in data[response][docs]:
                        web_links.append(item['web_url'])

now considering: 20150103 to 20150202
now considering: 20150202 to 20150304
now considering: 20150304 to 20150403
now considering: 20150403 to 20150503


In [118]:
# check the number of links
len(web_links)

30

In [119]:
# extract article text for the links obtained through nytimesarticle
for link in web_links:
    try:
        article = Article(link)
        article.download()
        article.parse()
        article_text = str(article.text).replace('\n','')
        df = pd.DataFrame([[article_text, 'movies']], columns=['article_text','category'])
        with open('./data/articles.csv', 'a') as f:
            df.to_csv(f, header=False, index=False)
        print("added to csv: " + link)
    except:
        print("could not download the link: " + link)
        continue

added to csv: https://www.nytimes.com/2015/01/23/movies/movie-listings-for-jan-23-29.html
added to csv: https://www.nytimes.com/2015/01/30/movies/movie-listings-for-jan-30-feb-5.html
added to csv: https://www.nytimes.com/video/movies/100000003481520/this-weeks-movies-jan-30-2015.html
added to csv: https://www.nytimes.com/2015/01/16/movies/movie-listings-for-jan-16-22.html
added to csv: https://www.nytimes.com/video/movies/100000003466832/this-weeks-movies-jan-23-2015.html
added to csv: https://www.nytimes.com/2015/01/09/movies/movie-listings-for-jan-9-15.html
added to csv: https://www.nytimes.com/video/movies/100000003453103/this-weeks-movies-jan-16-2014.html
added to csv: https://douthat.blogs.nytimes.com/2015/01/15/the-year-in-movies/
added to csv: https://www.nytimes.com/video/movies/100000003438692/this-weeks-movies-jan-9-2015.html
added to csv: https://www.nytimes.com/2015/01/17/opinion/when-movies-and-markets-clash.html
added to csv: https://www.nytimes.com/2015/03/01/magazine/le

In [55]:
#create a header for the file when creating the file
#head = pd.DataFrame([['article_text', 'category']], columns=['article_text','category'])
#with open('./data/articles.csv', 'a') as f:
#    head.to_csv(f, header=False, index=False)

In [135]:
# checking a few articles to see that we got it right
datafile = open('./data/articles.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
data = []
for row in datareader:
    data.append(row)    
for i in range(5):
    print(data[i])

['article_text', 'category']
['In recent years, I’ve caught a rather nasty case of replay anxiety, an increasingly common malady in American sports. The afflicted cannot watch a long touchdown run, a buzzer beater or a no-hitter reach the bottom of the ninth inning without worrying that the unfolding drama will be interrupted by the sight of umpires and referees trotting — always a bit too slowly and dutifully — to the replay monitors to review a call. Sitting on my couch, my eyes darting around the screen, I feel the strain of switching between two parallel realities: what the ref sees and what the machine knows.I feel bad for the refs, who, for the most part, are company men in an industry that desperately wants to replace their jobs with automatons. Management, in this case, seems determined to put the ref’s flaws on full display: Every time the ref makes a mistake — especially one that alters the outcome of a game — it’s played over and over again on the giant glowing Jumbotron tha

In [159]:
# imports required for shuffle
from random import shuffle 
import numpy as np

In [161]:
# shuffling the articles data
data = np.array(data)
np.random.shuffle(data)

In [166]:
# writing the shiffled articles back to a new csv
count = 0

for article, category in data:
    df = pd.DataFrame([[article, category]], columns=['article_text','category'])
    with open('./data/shuffled_articles.csv', 'a') as f:
            df.to_csv(f, header=False, index=False)

In [165]:
# creating header for the new csv before using it
#head = pd.DataFrame([['article_text', 'category']], columns=['article_text','category'])
#with open('./data/shuffled_articles.csv', 'a') as f:
#    head.to_csv(f, header=False, index=False)